ETL principal sobre los csv que se van a manipular una unica vez, los cuales van a estar subidos a cloud storage y se van a usar para combinarlos con los datos que se obtienen del web scraping

Lectura de archivos que están en csv e importación de librerías 

In [13]:
import pandas as pd
from collections import Counter
import ast
import numpy as np

#Importamos información de todos los datasets

#Importamos información de glassdoor
glassdor=pd.read_csv("Datasets_glassdoor/glassdoor.csv")

#Leemos los datos de reviews de glassdoor
review = pd.read_csv('Datasets_glassdoor/glassdoor_reviews.csv')

#Countries
countries= pd.read_csv("Datasets_glassdoor/cities_finales.csv", sep=";", encoding="latin-1")



Creacion de puestos de trabajo

In [4]:
PuestoTrabajo= pd.DataFrame()
NombrePuestoTrabajo=["data scientist", "data analyst", "data engineer"]
PuestoTrabajo["NombrePuesto"]= NombrePuestoTrabajo
PuestoTrabajo.to_csv("Datasets_glassdoor_limpio/PuestoTrabajo.csv", index=False)

Creación de herramientas

In [6]:
Herramientas= pd.DataFrame()
Herramientas_ds=pd.DataFrame({"herramienta":["SAS", "Apache Spark", "BigMl", 
                                             "D3.js", "MATLAB","Excel","ggplot2", 
                                             "Tableau", "Jupyter","Matplotlib", 
                                             "NLTK", "Scikit-learn", "TensorFlow", "Weka"]})

Herramientas_analytics=pd.DataFrame({"herramienta":["Microsoft Excel", "Python", "R",
                                                    "Jupyter Notebook","Apache Spark", 
                                                    "SAS","Microsoft Power BI", "Tableau", "KNIME"]})

Herramientas_engineer=pd.DataFrame({"herramienta":["Python", "SQL", "PostgreSQL", "MongoDB", 
                                                   "ApacheSpark", "Apache Kafka", "Amazon Redshift", 
                                                   "Snowflake", "Amazon Athena", "Apache Airflow", 
                                                   "Big Query", "Tableau", "Looker", "Apache Hive", 
                                                   "Segment", "Dbt", "Redash", "Fivetran", "Power BI", 
                                                   "Periscope", "Prefect", "Presto", "Metabase"]})

Herramientas= pd.concat([Herramientas_ds, Herramientas_analytics, Herramientas_engineer], axis=0)
Herramientas["herramienta"]= Herramientas["herramienta"].str.lower()

Herramientas.to_csv("Datasets_glassdoor_limpio/Herramientas.csv", index=False)

In [ ]:
Herramientas

,herramienta
0,sas
1,apache spark
2,bigml
3,d3.js
4,matlab
5,excel
6,ggplot2
7,tableau
8,jupyter
9,matplotlib


Creación de ofertas laborales

In [14]:
#Ofertas laborales glassdor
glassdoor= glassdor[["job.discoverDate", "job.description","salary.salaries", "map.country","gaTrackerData.empName","header.jobTitle",]]
glassdoor = glassdoor.assign(Modalidad = "presencial")
glassdoor = glassdoor.assign(PlataformasEmpleo= "glassdoor")

glassdoor = glassdoor.rename(columns = {"map.country":"pais",
                            "job.discoverDate":"Fecha",
                              "job.description":"DescripcionTrabajo",
                              "salary.salaries":"salario",
                              "gaTrackerData.empName":"NombreEmpresa",
                              "header.jobTitle":"PuestoTrabajo"})

#Unión de las tablas
ofertas_laborales= glassdoor


#Limpieza de columna PuestoTrabajo
#Pasamos todos los datos de la columna a minuscula

x= ofertas_laborales["PuestoTrabajo"].str.lower()
ofertas_laborales["PuestoTrabajo"]=x

#Creación de segmentación de ofertas laborales por perfil laboral

ofertas_solo_data=ofertas_laborales[ofertas_laborales.PuestoTrabajo.str.contains("data") | ofertas_laborales.PuestoTrabajo.str.contains("learning") | 
                                    ofertas_laborales.PuestoTrabajo.str.contains("analyst") | ofertas_laborales.PuestoTrabajo.str.contains("scientist")|
                                    ofertas_laborales.PuestoTrabajo.str.contains("Data")]


ofertas_data_scientist= ofertas_solo_data[ofertas_solo_data.PuestoTrabajo.str.contains("data scientist") | ofertas_solo_data.PuestoTrabajo.str.contains("learning") |
                                          ofertas_solo_data.PuestoTrabajo.str.contains("Scientist") ]
ofertas_data_scientist = ofertas_data_scientist.assign(puesto="data scientist")



ofertas_data_analyst= ofertas_solo_data[ofertas_solo_data.PuestoTrabajo.str.contains("data analyst") | ofertas_solo_data.PuestoTrabajo.str.contains("analyst") |
                                        ofertas_solo_data.PuestoTrabajo.str.contains("Analyst")]
ofertas_data_analyst = ofertas_data_analyst.assign(puesto="data analyst")

ofertas_data_engineer= ofertas_solo_data[ofertas_solo_data.PuestoTrabajo.str.contains("data engineer") | ofertas_solo_data.PuestoTrabajo.str.contains("warehouse") |
                                        ofertas_solo_data.PuestoTrabajo.str.contains("big data")|ofertas_solo_data.PuestoTrabajo.str.contains("data Engineer")]
ofertas_data_engineer = ofertas_data_engineer.assign(puesto="data engineer")

#Unimos las tablas segmentadas
ofertas_laborales_segmentadas= pd.concat([ofertas_data_analyst, ofertas_data_engineer, ofertas_data_scientist])


#Reacomodamos para el modelo relacional
ofertas_laborales_segmentadas=ofertas_laborales_segmentadas.drop(["PuestoTrabajo"], axis=1)
ofertas_laborales_segmentadas=ofertas_laborales_segmentadas.rename(columns={"puesto":"PuestoTrabajo"})


#Limpiamos la columna pais
#Definimos los códigos de países que nos servirán como guia
codigos_paises= countries["codigo_pais"].tolist()

#Identificamos los países que no cuentan con código internacional de país y los guardamos en un dataset
ofertas_laborales_limpiar= ofertas_laborales_segmentadas[ofertas_laborales_segmentadas["pais"].isin(codigos_paises)==False]

#Identificamos los países que cuentan con código internacional de país y los guardamos en otro dataset
ofertas_laborales_limpios= ofertas_laborales_segmentadas[ofertas_laborales_segmentadas["pais"].isin(codigos_paises)]


#Identificamos cuál es su código país de acuerdo al merge que hagamos con la tabla países
ofertas_laborales_limpiar= pd.merge(left=ofertas_laborales_limpiar, right= countries[["pais","codigo_pais"]], left_on="pais", right_on="pais", how="inner")

#Quitamos la columna de donde sacamos los datos sucios
ofertas_laborales_limpiar.drop(["pais"], axis="columns", inplace=True)

#Renombramos la columna país
ofertas_laborales_limpiar= ofertas_laborales_limpiar.rename(columns={"codigo_pais":"pais"})

#Unimos todo el dataframe, pero ahora sí con los datos de países limpios
ofertas_laborales_segmentadas=pd.concat([ofertas_laborales_limpios,ofertas_laborales_limpiar])

#Borramos los datos que no tengan país
ofertas_laborales_segmentadas= ofertas_laborales_segmentadas[ofertas_laborales_segmentadas["pais"].notna()]


#Ponemos en una lista todas las herramientas de análisis
lista_herramientas= Herramientas["herramienta"].tolist()

# Pasamos a minuscula el contenido
m= ofertas_laborales_segmentadas['DescripcionTrabajo'].str.lower()
m=pd.DataFrame(m)

##separar el contenido de la columna en palabras con split
m=m['DescripcionTrabajo'].str.split()

#Buscar en cada línea las palabras clave

lista = []
for i in m:
    guardo = []
    for p in lista_herramientas:
        if p in i:
            guardo.append(p)
    guardo=list(set(guardo))
    lista.append(guardo)
    
    
#Pasamos toda la información a la tabla de ofertas laborales segmentadas
ofertas_laborales_segmentadas["herramientas"]=lista

#Borramos la columna DescripcionTrabajo
ofertas_laborales_segmentadas.drop(["DescripcionTrabajo"], axis="columns", inplace=True)

#Limpieza de columna fecha
ofertas_laborales_segmentadas.Fecha= ofertas_laborales_segmentadas.Fecha.str.replace(r'T..+', "")
ofertas_laborales_segmentadas.Fecha=pd.to_datetime(ofertas_laborales_segmentadas.Fecha,infer_datetime_format=True)

#Nos aseguramos de no tener nulos
ofertas_laborales_segmentadas= ofertas_laborales_segmentadas[ofertas_laborales_segmentadas["NombreEmpresa"].notna()].reset_index(drop=True)

#Traemos la columna subregion
#Identificamos cuál es su código país de acuerdo al merge que hagamos con la tabla países
ofertas_laborales_segmentadas= pd.merge(left=ofertas_laborales_segmentadas, right= countries[["pais", "codigo_pais","sub_region"]], left_on="pais", right_on="codigo_pais", how="inner")

#Borramos pais_x
ofertas_laborales_segmentadas=ofertas_laborales_segmentadas.drop(["pais_x"], axis=1)

#Renombramos pais_y
ofertas_laborales_segmentadas.rename(columns = {"pais_y":"pais"
                                                }, inplace= True)

ofertas_laborales_segmentadas = ofertas_laborales_segmentadas.rename(columns={'salario':'Salario',
                                                                                'NombreEmpresa':'Empresa',
                                                                                'codigo_pais':'idPais',
                                                                                'sub_region':'Continente',
                                                                                'pais':'Pais'})
                                                                                
ofertas_laborales_segmentadas = ofertas_laborales_segmentadas[['Salario','Empresa','Modalidad','PlataformasEmpleo','Fecha','PuestoTrabajo','Pais','idPais','Continente','herramientas']]
ofertas_laborales_segmentadas.to_csv('Datasets_glassdoor_limpio/OfertasLaborales.csv')

C:\Users\sebastian\AppData\Local\Temp\ipykernel_4412\1281969715.py:109: FutureWarning: The default value of regex will change from True to False in a future version.
  ofertas_laborales_segmentadas.Fecha= ofertas_laborales_segmentadas.Fecha.str.replace(r'T..+', "")


In [11]:
ofertas_laborales_segmentadas

,Salario,Empresa,Modalidad,PlataformasEmpleo,Fecha,PuestoTrabajo,Pais,idPais,Continente,herramientas
0,2495.0,Amaris,presencial,glassdoor,2019-10-29,data analyst,Belgium,BE,Western Europe,[r]
1,2544.0,PYCO Group,presencial,glassdoor,2019-11-01,data analyst,Belgium,BE,Western Europe,"[sql, tableau]"
2,2733.0,Unique,presencial,glassdoor,2019-11-06,data analyst,Belgium,BE,Western Europe,[excel]
3,7543.0,VOO,presencial,glassdoor,2019-10-29,data analyst,Belgium,BE,Western Europe,"[python, sql, tableau]"
4,7748.0,"Stubbe, Lisart & Partners",presencial,glassdoor,2019-10-15,data analyst,Belgium,BE,Western Europe,[]
...,...,...,...,...,...,...,...,...,...,...
24817,4257.0,CACI International,presencial,glassdoor,2019-10-30,data analyst,Kuwait,KW,Western Asia,[]
24818,65421.0,Real Estate Board of Jamaica,presencial,glassdoor,2019-10-26,data analyst,Jamaica,JM,Latin America and the Caribbean,[sql]
24819,126586.0,toters,presencial,glassdoor,2019-10-24,data analyst,Lebanon,LB,Western Asia,[]
24820,145951.0,Fairmont Hotels & Resorts,presencial,glassdoor,2019-10-21,data analyst,Azerbaijan,AZ,Western Asia,[]


Creacion de herramientas por perfil

In [16]:
#Filtramos la carrera de data science, data analyst, data engineer
#data science


herr_data_science= ofertas_laborales_segmentadas[ofertas_laborales_segmentadas["PuestoTrabajo"]=="data scientist"]
herr_ds=herr_data_science["herramientas"].tolist()
lista_plana = []

for elemento in herr_ds:
    for item in elemento:
        lista_plana.append(item)
        
lista=Counter(lista_plana)

herr_ds=pd.DataFrame.from_dict(lista, orient="index").reset_index()
herr_ds["PuestoTrabajo"]="data scientist"


#data engineer
herr_data_engineer= ofertas_laborales_segmentadas[ofertas_laborales_segmentadas["PuestoTrabajo"]=="data engineer"]
herr_de=herr_data_engineer["herramientas"].tolist()
lista_plana = []
for elemento in herr_de:
    for item in elemento:
        lista_plana.append(item)

lista=Counter(lista_plana)
herr_de=pd.DataFrame.from_dict(lista, orient="index").reset_index()
herr_de["PuestoTrabajo"]="data engineer"


#data analyst
herr_data_analyst= ofertas_laborales_segmentadas[ofertas_laborales_segmentadas["PuestoTrabajo"]=="data analyst"]
herr_da=herr_data_analyst["herramientas"].tolist()
lista_plana = []
for elemento in herr_da:
    for item in elemento:
        lista_plana.append(item)

lista=Counter(lista_plana)
herr_da=pd.DataFrame.from_dict(lista, orient="index").reset_index()
herr_da["PuestoTrabajo"]="data analyst"

#unimos todo el dataframe en herramientas por carrera
herr_carrera= pd.concat([herr_ds, herr_de, herr_da])

#Renombramos columnas
herr_carrera= herr_carrera.rename(columns={"index":"herramientas",
                             0:"cuenta",     
})

#Enviamos los datos a un csv
herr_carrera.to_csv("Datasets_glassdoor_limpio\herr_carrera.csv", index=False)

In [17]:
herr_carrera

,herramientas,cuenta,PuestoTrabajo
0,python,1559,data scientist
1,sql,906,data scientist
2,r,844,data scientist
3,sas,172,data scientist
4,tensorflow,194,data scientist
...,...,...,...
20,metabase,6,data analyst
21,tensorflow,4,data analyst
22,mongodb,17,data analyst
23,matplotlib,1,data analyst


Plataformas de empleo

In [19]:
plat_empleo = pd.DataFrame()
lista = ['glassdoor','linkedin']
plat_empleo['PlataformasEmpleo'] = lista
plat_empleo.to_csv('Datasets_glassdoor_limpio/PlataformasEmpleo.csv',index=False)

Modalidad

In [20]:
modalidad=pd.DataFrame(ofertas_laborales_segmentadas["Modalidad"].drop_duplicates()).reset_index(drop=True)
modalidad.to_csv("Datasets_glassdoor_limpio/Modalidad.csv")   

Creacion tabla empresa pais

In [21]:
empresa_pais= ofertas_laborales_segmentadas[["Empresa", "idPais", "Continente"]].drop_duplicates()
empresa_pais.to_csv("Datasets_glassdoor_limpio/empresa_pais.csv", index= False)

In [22]:
empresa_pais

,Empresa,idPais,Continente
0,Amaris,BE,Western Europe
1,PYCO Group,BE,Western Europe
2,Unique,BE,Western Europe
3,VOO,BE,Western Europe
4,"Stubbe, Lisart & Partners",BE,Western Europe
...,...,...,...
24816,CACI International,KW,Western Asia
24818,Real Estate Board of Jamaica,JM,Latin America and the Caribbean
24819,toters,LB,Western Asia
24820,Fairmont Hotels & Resorts,AZ,Western Asia


Ratings empresa

In [24]:


#Seleccionamos las columnas de interés
review_clean = review.drop(columns = ["reviews.val.companyResponse","reviews.val.reviewResponses","reviews.val.adviceToManagement","reviews.val.title","reviews.val.publishedOn","reviews.val.reviewerStatus"])

review_clean.dropna(inplace=True)


review_final= review_clean[["id","reviews.val.reviewRatings.careerOpportunities", "reviews.val.reviewRatings.compBenefits", "reviews.val.reviewRatings.cultureValues", 
"reviews.val.reviewRatings.overall", "reviews.val.reviewRatings.seniorManagement", "reviews.val.reviewRatings.worklifeBalance", "reviews.val.summaryPoints.ceoApproval",
"reviews.val.summaryPoints.outlook", "reviews.val.summaryPoints.recommend"]]

review_final= review_final.rename(columns={"empresa": "enterprise","reviews.val.reviewRatings.careerOpportunities":"rateCareerOpportunities", "reviews.val.reviewRatings.compBenefits": "rateCompensation_Benefits",
                                 "reviews.val.reviewRatings.cultureValues":"rateCultureValues", "reviews.val.reviewRatings.overall":"rateGeneralView",
                                 "reviews.val.reviewRatings.seniorManagement":"rateOpinionAboutManagement", "reviews.val.reviewRatings.worklifeBalance":"rateWorklifeBalance",
                                 "reviews.val.summaryPoints.ceoApproval":"rateCeoApproval","reviews.val.summaryPoints.outlook":"ratePositiveBusinessView",
                                  "reviews.val.summaryPoints.recommend":"rateRecommendToFriend"})


#Construimos la tabla empresa por id
glassdor_empresas=glassdor[["benefits.highlights", "gaTrackerData.empName"]].drop_duplicates(subset="gaTrackerData.empName")

#Juntamos el nombre de empresa con el id de la tabla benefits
review_final= pd.merge(left=review_final, right= glassdor_empresas, left_on="id", right_on="benefits.highlights", how="inner")

#Juntamos el pais, region por empresa
review_final=pd.merge(left=review_final, right= empresa_pais, left_on="gaTrackerData.empName", right_on="Empresa", how="inner")

#Borro columnas repetidas
review_final=review_final.drop(["id","benefits.highlights","gaTrackerData.empName"], axis=1).reset_index(drop=True)

review_final.to_csv("Datasets_glassdoor_limpio/review_empresa.csv")


In [25]:
review_final

,rateCareerOpportunities,rateCompensation_Benefits,rateCultureValues,rateGeneralView,rateOpinionAboutManagement,rateWorklifeBalance,rateCeoApproval,ratePositiveBusinessView,rateRecommendToFriend,Empresa,idPais,Continente
0,3.0,4.0,5.0,4.0,4.0,4.0,1.0,1.0,1.0,Frazer-Nash Consultancy,GB,Northern Europe
1,4.0,3.0,4.0,3.0,2.0,2.0,3.0,3.0,2.0,Frazer-Nash Consultancy,GB,Northern Europe
2,4.0,2.0,2.0,2.0,1.0,3.0,2.0,3.0,2.0,Frazer-Nash Consultancy,GB,Northern Europe
3,4.0,3.0,4.0,4.0,4.0,5.0,1.0,1.0,1.0,Frazer-Nash Consultancy,GB,Northern Europe
4,5.0,4.0,5.0,5.0,5.0,5.0,1.0,1.0,1.0,AffiniPay,US,Northern America
...,...,...,...,...,...,...,...,...,...,...,...,...
14867,4.0,3.0,4.0,5.0,4.0,4.0,3.0,2.0,1.0,Ignite Technical Resources,CA,Northern America
14868,3.0,3.0,3.0,4.0,3.0,3.0,1.0,1.0,1.0,Ignite Technical Resources,CA,Northern America
14869,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,Ignite Technical Resources,CA,Northern America
14870,4.0,3.0,4.0,4.0,4.0,5.0,3.0,2.0,1.0,Ignite Technical Resources,CA,Northern America


Tabla continentes

In [26]:
continentes= pd.DataFrame(countries["sub_region"].drop_duplicates().reset_index(drop=True))
continentes.to_csv("Datasets_glassdoor_limpio\continentes.csv")

In [ ]:
continentes

,sub_region
0,Southern Asia
1,Northern Europe
2,Southern Europe
3,Northern Africa
4,Polynesia
5,Sub-Saharan Africa
6,Latin America and the Caribbean
7,Antarctica
8,Western Asia
9,Australia and New Zealand
